In [2]:
%run ChanBiManager.ipynb
%run Util.ipynb

# 一个波段笔的情况
class BiInSectionService:
    
    def __init__(self,symbol,level,short,mid):
        self.symbol = symbol
        self.level = level
        self.short = short
        self.mid = mid
        
        self.context = {}
        
        self.context['symbol'] = symbol
        self.context['level'] = level
        self.context['short'] = short
        self.context['mid'] = mid
        
        self.chan_bi_analyser = ChanBiManager().get_analyser(symbol,level,short,mid)
        
    def get_chan_bi_df_in_section(self,section_start, section_end):
        chan_bi_df = self.chan_bi_analyser.get_chan_bis_between(section_start, section_end)
        return chan_bi_df
        
        
    def the_first_bi_info(self,section):
        section_start = section['d_s_date']
        section_end = section['d_e_date']
        section_start_price = section['d_s_price']
        
        chan_bi_df = self.get_chan_bi_df_in_section(section_start, section_end)
        
        if(chan_bi_df.shape[0]<1):
            return
        
        the_first_bi = chan_bi_df.iloc[0]
        the_second_bi = chan_bi_df.iloc[1]
        
        first_bi_direc  = the_first_bi['turning_point']
        if(first_bi_direc=='bottom'):
            first_bi_direc = 'up'
        else:
            first_bi_direc = 'down'
        
        price_range = round(100* (the_second_bi['price']-section_start_price)/section_start_price,2)
        
        return [self.symbol,section_start,section_end,first_bi_direc,price_range]
        
            
    def the_first_bi_price_probability_distribution(self):
        section_bi_df = pd.DataFrame(columns=['symbol','section_start','section_end','first_bi_direc','first_bi_price_range'])
        
        for key,row in section_df.iterrows():
            bi_info = self.the_first_bi_info(row)
            if(bi_info is not None):
                section_bi_df.loc[section_bi_df.shape[0]+1] = bi_info
        
        return section_bi_df
    
    
    def is_the_enter_k_chan_bi_direct_up(self,section,chan_bi_df):
        section_start = section['d_s_date']
        section_end = section['d_e_date']
        section_start_price = section['d_s_price']
        
        if(chan_bi_df.shape[0]<1):
            return False
        
        the_first_bi = chan_bi_df.iloc[0]
    
        return the_first_bi['turning_point']=='bottom'
    
    def does_break_the_enter(self,section):
        section_start = section['s_date']
        section_end = section['e_date']
        section_start_price = section['s_price']
        
        chan_bi_df = self.get_chan_bi_df_in_section(section_start, section_end)
        
        if(chan_bi_df.shape[0]<1):
            return
        
        the_first_bi = chan_bi_df.iloc[0]
        the_second_bi = chan_bi_df.iloc[1]
        
        
    def break_the_enter_point_distribution(self,section_df):
        break_point_df = pd.DataFrame(columns=chan_bi_in_section_columns)
        
        for key,row in section_df.iterrows():
            section_start = row['d_s_date']
            section_end = row['d_e_date']
            section_start_price = row['d_s_price']
            section_end_price = row['d_e_price']

            chan_bi_df = self.get_chan_bi_df_in_section(section_start, section_end)
            if(chan_bi_df.shape[0]<=4):
                continue

            first_k_direct_up = self.is_the_enter_k_chan_bi_direct_up(row,chan_bi_df)
            breaked=False
            
            if(first_k_direct_up==True):
                the_second_up_bi = chan_bi_df[chan_bi_df['turning_point']=='bottom'].iloc[1]

                if(the_second_up_bi['price']<section_start_price):
                    breaked=True
            else:
                bottom_bi_df = chan_bi_df[chan_bi_df['turning_point']=='bottom']
                the_first_down_bi = bottom_bi_df.iloc[0]
                the_second_down_bi =  bottom_bi_df.iloc[1]
                
                if(the_second_down_bi['price']<the_first_down_bi['price']):
                    breaked=True
                    
            break_point_df.loc[break_point_df.shape[0]+1]=[row['symbol'],section_start,section_end,section_start_price,section_end_price,first_k_direct_up,breaked]
        
        return break_point_df
    
    
    def get_context(self):
        return self.context
        
    
    